In [1]:
# import numpy as np
# import pandas as pd


In [2]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import numpy as np
import pandas as pd
import random

In [4]:
# create data set
key_1 = [101]*100
key_2 = [201]* 700000
key_3 = [301] * 500
key_4 = [401] *1000
orderID = key_1 + key_2 +key_3 +key_4
random.shuffle(orderID)

qty_1=    list(np.random.randint(low = 1,high = 100,size = len(key_1)))
qty_2 =   list(np.random.randint(low = 1,high = 200,size = len(key_2))) 
qty_3 = list(np.random.randint(low = 1,high = 1000,size = len(key_3)))
qty_4 = list(np.random.randint(low = 1,high = 50,size = len(key_4)))
qty = qty_1+qty_2+qty_3+qty_4

sales_1 = list(np.random.randint(low  = 10,high = 100,size = len(key_1)))
sales_2 = list(np.random.randint(low = 50 ,high = 3400 ,size = len(key_2)))
sales_3 = list(np.random.randint(low = 12 ,high =2000 ,size = len(key_3)))
sales_4 = list(np.random.randint(low = 40 ,high = 1000,size = len(key_4)))
sales = sales_1 + sales_2+sales_3 +sales_4
discount = list(np.random.randint(low  =0,high = 2,size = len(orderID)))
data1 = list(zip(orderID,qty,sales,discount))


In [5]:
# create pandas dataframe
data_skew = pd.DataFrame(data1,columns = ['orderID','qty','sales','price'])

In [6]:
# small data 

data2 = [[1,101,'pencil',4.99],
         [2,101,'book',9.5],
         [3,101,'scissors',14],
         [4,301,'glue',7],
         [5,201,'marker',8.49],
         [6,301,'label',2],
         [7,201,'calculator',3.99],
         [8,501,'eraser',1.55]
    
    
]
data_small = pd.DataFrame(data2,columns = ['productID','orderID','product','price'])

In [8]:
# convert pandas to spark dataframe 

# spark.conf.set('spark.sql.execution.arrow.enabled','true')

In [ ]:
print(spark.conf.get('spark.sql.execution.arrow.enabled'))

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("skew_df").getOrCreate()
spark

24/04/12 20:16:46 WARN Utils: Your hostname, arphaxad-HP-Notebook resolves to a loopback address: 127.0.1.1; using 192.168.1.4 instead (on interface wlp3s0)
24/04/12 20:16:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/12 20:16:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
print(spark.conf.get('spark.sql.execution.arrow.pyspark.enabled'))

In [10]:
# convert pandas to spark dataframe 

spark.conf.set('spark.sql.execution.arrow.enabled','true')
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled','true')

24/04/12 20:22:02 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [11]:
df_skew = spark.createDataFrame(data_skew)
df_skew.printSchema()
df_skew.rdd.getNumPartitions()

/home/arphaxad/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:351: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 4.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


root
 |-- orderID: long (nullable = true)
 |-- qty: long (nullable = true)
 |-- sales: long (nullable = true)
 |-- price: long (nullable = true)



4

In [12]:
df_small  = spark.createDataFrame(data_small)
df_small.printSchema()
df_small.rdd.getNumPartitions()

root
 |-- productID: long (nullable = true)
 |-- orderID: long (nullable = true)
 |-- product: string (nullable = true)
 |-- price: double (nullable = true)



4

In [13]:
# df_skew.show()
df_skew = df_skew.repartition(8)
df_small = df_small.repartition(8)

In [25]:
#spark.conf.set("spark.sql.adaptive.enabled", "false")...............................................

In [24]:
df_small.rdd.getNumPartitions()

8

In [ ]:
joined_df = df_skew.join(df_small,df_skew.orderID==df_small.orderID,how = "inner")


In [ ]:
joined_df.count()

In [23]:
joined_df.rdd.getNumPartitions()

NameError: name 'joined_df' is not defined

In [ ]:
# joined_df.(4)
joined_df.rdd.getNumPartitions()

In [ ]:
print(spark.conf.get("spark.sql.shuffle.partitions"))

In [ ]:
joined_df.rdd.glom().collect()

In [20]:
spark.conf.set("spark.sql.shuffle.partitions",8)

In [28]:
from pyspark.sql.functions import *
groups = df_skew.join(df_small,df_skew.orderID == df_small.orderID, how="inner")
summary = groups.select(mean(groups.sales).alias("avg_sales"),
                       stddev(groups.sales).alias("std_dev"),
                        min(groups.sales).alias("min sales"),
                        max(groups.sales).alias("max sales")
                       )
summary.show()

24/04/12 20:30:23 WARN TaskSetManager: Stage 47 contains a task of very large size (2053 KiB). The maximum recommended task size is 1000 KiB.


+------------------+-----------------+---------+---------+
|         avg_sales|          std_dev|min sales|max sales|
+------------------+-----------------+---------+---------+
|1721.5427024905446|967.4304609758726|       10|     3399|
+------------------+-----------------+---------+---------+



In [ ]:
from pyspark.sql.functions import *
df_skew_salt = df_skew.withColumn("salt",round(rand()*2))
df_small_salt = df_small.withColumn("salt",round(rand()*2))
                      
                                  

In [27]:
groups.rdd.getNumPartitions()

8

In [41]:
#salting


spark.conf.set("spark.sql.shuffle.partitions",8)
from pyspark.sql.functions import *
df_skew_salt = df_skew.withColumn("salt",round(rand()*7))
df_small_salt = df_small.withColumn("salt",round(rand()*27))
                      
df_skew_salt = df_skew_salt.repartition(8,"salt")
df_small_salt = df_small_salt.repartition(8,"salt")
groups = df_skew.join(df_small,df_skew.orderID == df_small.orderID, how="inner")
summary = groups.select(mean(groups.sales).alias("avg_sales"),
                       stddev(groups.sales).alias("std_dev"),
                        min(groups.sales).alias("min sales"),
                        max(groups.sales).alias("max sales")
                       )
summary.show()

24/04/13 00:17:25 WARN TaskSetManager: Stage 63 contains a task of very large size (2053 KiB). The maximum recommended task size is 1000 KiB.


+------------------+-----------------+---------+---------+
|         avg_sales|          std_dev|min sales|max sales|
+------------------+-----------------+---------+---------+
|1721.5427024905446|967.4304609758591|       10|     3399|
+------------------+-----------------+---------+---------+



In [44]:
summary.rdd.getNumPartitions()

24/04/13 00:18:07 WARN TaskSetManager: Stage 76 contains a task of very large size (2053 KiB). The maximum recommended task size is 1000 KiB.


1

In [ ]:
#repsrtition
df_skew_salt = df_skew_salt.repartition(3,"salt")
df_small_salt = df_small_salt.repartition(3,"salt")

In [ ]:
df_skew_salt.rdd.getNumPartitions()

In [ ]:
from pyspark.sql.functions import *
groups = df_skew_salt.join(df_small_salt,df_skew_salt.orderID == df_small_salt.orderID, how="inner")
summary = groups.select(mean(groups.sales).alias("avg_sales"),
                       stddev(groups.sales).alias("std_dev"),
                        min(groups.sales).alias("min sales"),
                        max(groups.sales).alias("max sales")
                       )
summary.show()

In [33]:
from pyspark.sql.functions import *
groups_brd = df_skew.join(broadcast(df_small),df_skew.orderID == df_small.orderID,how = "inner")
summ = groups_brd.select(mean(groups_brd.sales).alias("avf sales"),
                        stddev(groups_brd.sales).alias("std dev"),
                         min(groups_brd.sales).alias("min sales"),
                         max(groups_brd.sales).alias("max sales"))
summ.show()

24/04/12 23:53:28 WARN TaskSetManager: Stage 60 contains a task of very large size (2053 KiB). The maximum recommended task size is 1000 KiB.


+------------------+-----------------+---------+---------+
|         avf sales|          std dev|min sales|max sales|
+------------------+-----------------+---------+---------+
|1721.5427024905446|967.4304609758585|       10|     3399|
+------------------+-----------------+---------+---------+



In [37]:
print((spark.conf.get("spark.sql.files.maxPartitionBytes")))

134217728b


In [38]:
134217728/1024/1024

128.0

In [40]:
spark.conf.set("spark.sql.adaptive.enabled","true")